In [1]:
# coding: utf-8

# please change min_count=5, dm=0 parameters as you wish to replicate an experiment
# train test split is  %60 to %40 

data_path = "DATASTORE/DatasetA/corpus"  #where training data are (corpus)
path_train = "DATASTORE/DatasetA/train"  # where training user tweets are
path_test = "DATASTORE/DatasetA/test"    # where training user tweets are

In [2]:
endItr = 100  #last iteration 
max_epochs = endItr

from multiprocessing import cpu_count
workers = cpu_count() * 0.75

vec_size = 100
alpha = 0.025

In [4]:
#Import all  dependencies
from sklearn import metrics

import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import os
import pandas as pd
import numpy as np
import sys
from datetime import datetime 
from scipy import spatial
#result = 1 - spatial.distance.cosine(v1, v2)

from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

here = os.getcwd()
import json
import pathlib

from reviewFunctions import user2vec_test

import logging
logging.basicConfig(filename="doc2vec_DatasetA.log",
                    filemode='a',
                    format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.CRITICAL)

logging.critical("\n doc2vec_DatasetA experiment 1")
logger = logging.getLogger('doc2vec_DatasetA')

stop_words = set(stopwords.words('english'))


#Data cleaning
def clean_tokenize(text):
    text = text.lower()
    text = word_tokenize(text)
    word_list = []
    for i in text:      
        if (i.isalnum() and (not i in stop_words)):
            word_list += [i]
        
    return word_list

def docIter(data_path):
    i = -1
    for root, dirs, files in os.walk(data_path):
        for file in files:
            print(file)
            file_path = os.path.join(root, file)
            document = open(file_path, encoding="utf-8").readlines()
            for line in document:
                i += 1
                yield gensim.models.doc2vec.TaggedDocument(clean_tokenize(line), [i])

def get_user_dataframe(path):
    dataFrames = []
    for folder in os.listdir(path):
        if os.path.isdir(os.path.join(path, folder)):
            print("folder: ", folder)
            for file_name in os.listdir(os.path.join(path, folder)):
                if file_name.endswith(".csv"):
                    file_path = os.path.join(path, folder, file_name)
                    df = pd.read_csv(file_path,header=None,usecols=[3,0,2], 
                                     names=['tweet_id', 'date', 'user_name', 'text'])
                    df = df.astype(str)
                    df["category"] = folder
                    dataFrames.append(df)

    dfs = pd.concat(dataFrames)
    print("total ", len(dfs), " tweets")
    return dfs



#saveJson(path_prefix, user_vecs, ctg_vecs, self.epoch) changedd
def saveJson(path_prefix, user_vecs_test, ctg_vecs_train, user_vecs_train, epoch):
    logging.critical("saveJSon started")
    path = get_tmpfile('{}/embeddings/users_test{}.json'.format(path_prefix,epoch))
    with open(path, 'w') as outfile:
        json.dump(user_vecs_test, outfile)
    outfile.close()

    path = get_tmpfile('{}/embeddings/categories_train{}.json'.format(path_prefix,epoch))
    with open(path, 'w') as outfile:
        json.dump(ctg_vecs_train, outfile)
    outfile.close()

    path = get_tmpfile('{}/embeddings/users_train{}.json'.format(path_prefix,epoch))
    with open(path, 'w') as outfile:
        json.dump(user_vecs_train, outfile)
    outfile.close()
    logging.critical("saveJSon finished")

/vol/ibrahim/.conda/envs/py37irh/bin/python


In [5]:
train_data = list(docIter(data_path))
dfs_train = get_user_dataframe(path_train)
dfs_test = get_user_dataframe(path_test)

datasetA_tweetOnly.txt
folder:  twcollector3
folder:  twcollector5
folder:  twcollector1
folder:  twcollector2
folder:  twcollector4
total  300000  tweets
folder:  twcollector3
folder:  twcollector5
folder:  twcollector1
folder:  twcollector2
folder:  twcollector4
total  200000  tweets


In [6]:
def calc_iter_acc(model, dfs_train, dfs_test, itr):
    # creaa
    user_vecs_train = {}
    ctg_vecs_train = {}
    user_vecs_test = {}
  
    logging.critical("beginning: of calc_iter_acc")
    logging.critical("first: inferring user_vecs_train")
    for index, datapoint in dfs_train.iterrows():
        unname = datapoint["user_name"]
        tid = datapoint["tweet_id"]
        category = datapoint["category"]
        tokenized_words = clean_tokenize(datapoint["text"])
        vec = model.infer_vector(tokenized_words)
        if unname in user_vecs_train.keys():
            user_vecs_train[unname]["vecs"].append(vec)
        else:
            user_vecs_train[unname] = {"vecs": [vec], "category": category}

    # find the average of tweet vectors for each user
    for unm in user_vecs_train.keys():
        user_vecs_train[unm]["avr_vec"] = np.average(np.array(user_vecs_train[unm]["vecs"]), axis=0)              
      
    #
    logging.critical("second: inferring user_vecs_test")
    for index, datapoint in dfs_test.iterrows():
        unname = datapoint["user_name"]
        tid = datapoint["tweet_id"]
        category = datapoint["category"]
        tokenized_words = clean_tokenize(datapoint["text"])
        vec = model.infer_vector(tokenized_words)
        if unname in user_vecs_test.keys():
            user_vecs_test[unname]["vecs"].append(vec)
        else:
            user_vecs_test[unname] = {"vecs": [vec], "category": category}

    # find the average of tweet vectors for each user, dfs_test
    for unm in user_vecs_test.keys():
        user_vecs_test[unm]["avr_vec"] = np.average(np.array(user_vecs_test[unm]["vecs"]), axis=0)
    #
    
    # create category vector dictionary
    for unm in user_vecs_train.keys():
        avg = user_vecs_train[unm]["avr_vec"]
        ctg = user_vecs_train[unm]["category"]
        if ctg in ctg_vecs_train.keys():
            ctg_vecs_train[ctg]["cat_vecs"].append(avg)
        else:
            ctg_vecs_train[ctg] = {"cat_vecs": [avg]}

    # find the average of category vectors
    for ctg in ctg_vecs_train.keys():
        ctg_vecs_train[ctg]["avr_cat_vec"] = np.average(np.array(ctg_vecs_train[ctg]["cat_vecs"]), axis=0)  
    
    users_test = {}
    for usr in user_vecs_test.keys():
        users_test[usr] = {'avr_vec' : user_vecs_test[usr]["avr_vec"].tolist(), 'category': user_vecs_test[usr]["category"]}
    
    users_train = {}
    for usr in user_vecs_train.keys():
        users_train[usr] = {'avr_vec' : user_vecs_train[usr]["avr_vec"].tolist(), 'category': user_vecs_train[usr]["category"]}
    
    
    categories_train = {}
    for ctg in ctg_vecs_train.keys():
        categories_train[ctg] = {'avr_cat_vec' : ctg_vecs_train[ctg]["avr_cat_vec"].tolist()}
    
    logging.critical("end of: calc_iter_acc")
    uvt = user2vec_test(user_vecs=users_test, ctg_vecs=categories_train) # test model
    msg1 = uvt.calc_accuracy()
    logging.critical(msg1)
    msg2 = uvt.calc_accuracy_by_group()
    logging.critical(msg2)
   
    #conf matrix
    true_labels = []
    predicted_labes = []

    for unm in uvt.user_vecs.keys():
        true_labels.append(uvt.getUserCategory(unm))
        predicted_labes.append(uvt.most_similar_group(unm))

    # Print the confusion matrix
    msg3 = metrics.confusion_matrix(true_labels, predicted_labes)
    logging.critical(msg3)

    # Print the precision and recall, among other metrics
    msg4 = metrics.classification_report(true_labels, predicted_labes, digits=3)
    logging.critical(msg4)
    
    subject = "doc2vec GPU2 alert - Epoch_"+ str(itr) + " Acc and Conf Matrix"
    #send_email(user, pwd, recipient, subject, msg1 + "\n\n" + msg2 + "\n\n" + msg4)     
    return users_test, categories_train, users_train, uvt.acc

class EpochSaver(CallbackAny2Vec):
#'''Asses the model at each iteration.'''
    def __init__(self, path_prefix):
        print("__init-EpochSaver__ called")
        self.path_prefix = path_prefix
        self.epoch = 1
        self.best_acc = 0
        self.best_epoch = 1
        msg = 'Epoch-{} started'.format(self.epoch)
        logging.critical(msg)

    def on_epoch_end(self, model):
        msg = 'Epoch-{} finished \n'.format(self.epoch)
        logging.critical(msg)
        # calc_iter_acc returns: users_test, categories_train, users_train, uvt.acc
        user_vecs, ctg_vecs, user_vecs_train, tmp_acc  = calc_iter_acc(model, dfs_train, dfs_test, self.epoch)
        if tmp_acc > self.best_acc:
            msg = "model improved as general acc  \n"
            print(msg)
            logging.critical(msg)
            self.best_acc = tmp_acc
            self.best_epoch = self.epoch            
            model_path = get_tmpfile('{}/embeddings/epoch{}.model'.format(self.path_prefix, self.epoch))
            model.save(model_path) 
        else:
            msg = "not saving the model Bc it didn't improve as general_acc  \n"
            print(msg)
            logging.critical(msg)             
        #saveJson(path_prefix, user_vecs_test, ctg_vecs_train, user_vecs_train, epoch
        saveJson(self.path_prefix, user_vecs, ctg_vecs, user_vecs_train, self.epoch)  
        self.epoch += 1

In [7]:
here = os.getcwd()
epoch_saver = EpochSaver(path_prefix=here)

__init-EpochSaver__ called


In [8]:
model = Doc2Vec(vector_size=vec_size, min_count=5,alpha=alpha,min_alpha=0.00025,dm=0, 
                epochs=max_epochs, callbacks=[epoch_saver], workers=workers)

In [9]:
model.build_vocab(train_data)

In [ ]:
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)
print("End of training")

model improved as general acc  



/vol/ibrahim/.conda/envs/py37irh/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
logging.critical("End of the experiment")